In [3]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from scipy.spatial.distance import cosine
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import sys

In [4]:
# extract a single face from an uploaded photograph
detector = MTCNN()
def extract_face(filename,detector=detector,required_size=(224, 224)):
	pixels = plt.imread(filename)
	# create the detector, using default weights
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array
    # resize pixels to the model size
  
# create a vggface model
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

#the resnet model represents the face in a 2048-dimension feature space.

def get_embedding(filename,model=model):
    # extract faces
    face = extract_face(filename)
    # convert into an array of samples
    sample = [asarray(face, 'float32')]
    # prepare the face for the model, e.g. center pixels
    sample = preprocess_input(sample, version=2)
    # perform prediction
    yhat = model.predict(sample)
    return yhat

def is_match(ID_embedding, subject_embedding, show_faces,thresh=0.4):
    # calculate distance between embeddings
    score = cosine(ID_embedding, subject_embedding)
    res = {'verified' : score <= 0.4, 'distance': score}

def face_classification(img1, img2):
  embedding1 = get_embedding(img1)
  embedding2 = get_embedding(img2)
  return is_match(embedding1,embedding2)

In [5]:
folderPath = 'drive/MyDrive/FGNET/images'

In [6]:
import os
os.path.exists(folderPath)

True

In [7]:
import pandas as pd

In [9]:
main_dic = dict()
folders = os.listdir(folderPath)
folders.sort()
folders = folders[:51]
for folder in folders:
  files = os.listdir(f"{folderPath}/{folder}")
  files.sort()
  image1 = f"{folderPath}/{folder}/{files[2]}"

  print(f"Person {folder}")

  df_dict = dict()
  df_dict[files[2][4:-4]] = 0
  for i in range(3, len(files)):
    res = face_classification(image1, f"{folderPath}/{folder}/{files[i]}")
    
    df_dict[files[i][4:-4]] = res['distance']

    print(f"Age 1st:{files[2][4:-4]} Age 2nd: {files[i][4:-4]}, verified: {res['verified']}, cosine distance: {res['distance']}")

  print(df_dict)
  main_dic[files[2][:3]] = df_dict
  print(main_dic)
  print()
  print()

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 44ms/step
Age 1st:05 Age 2nd: 20, verified: True, cosine distance: 0.3650434273453318
1/1 [==============================] - 0s 28ms/step
Age 1st:05 Age 2nd: 22, verified: False, cosine distance: 0.4963013850414061
1/1 [==============================] - 0s 28ms/step
Age 1st:05 Age 2nd: 23, verified: False, cosine distance: 0.4559413494106954
1/1 [==============================] - 0s 33ms/step
Age 1st:05 Age 2nd: 29, verified: False, cosine distance: 0.4664124761805657
1/1 [==============================] - 0s 56ms/step
Age 1st:05 Age 2nd: 31, verified: False, cosine distance: 0.4267915679029609
1/1 [==============================] - 0s 30ms/step
Age 1st:05 Age 2nd: 33, verified: False, cosine distance: 0.4780328087721173
{'05': 0, '10': 0.36437088050482636, '11': 0.43977170677415567, '13': 0.2845144104796373, '16': 0.3908169673684566, '20': 0.3650434273453318, '22': 0.4963013850414061, '23': 0.

In [10]:
print(main_dic)

{'001': {'08': 0, '10': 0.15917173212387092, '14': 0.322313456466781, '16': 0.29240770392094173, '18': 0.44541380364631633, '19': 0.40452902921426004, '22': 0.34814981524053923, '28': 0.2721238628435688, '29': 0.3088517471219622, '33': 0.38429077107587073, '40': 0.46335924627638125, '43a': 0.47507719728820474, '43b': 0.4524556241291585}, '002': {'05': 0, '07': 0.13933748312851424, '12': 0.2500832156703908, '15': 0.30685683323345514, '16': 0.3147819416585621, '18': 0.3233150929639237, '20': 0.22434840012469293, '21': 0.4046713617478209, '23': 0.47671376293289225, '26': 0.4200054471801621, '29': 0.3635863544722786, '31': 0.3657926931859573, '36': 0.3140313683513607, '38': 0.3370174192553185}, '003': {'23': 0, '25': 0.1791624690157928, '35': 0.21408911970026467, '38': 0.2507973043858095, '47': 0.30428476676942673, '49': 0.24984161522519432, '51': 0.34749695323305974, '58': 0.3969750965176353, '60': 0.3552138206306782, '61': 0.34584603437097017}, '004': {'26': 0, '28': 0.2590506549679127, 

In [11]:
df =  pd.DataFrame(main_dic)
df = df.sort_index()

In [12]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [13]:
df

,001,002,003,004,005,006,007,008,009,010,011,012,013,014,015,016,017,018,019,020,021,022,023,024,025,026,027,028,029,030,031,032,033,034,035,036,037,038,039,040,041,042,043,044,045,046,047,048,049,050,051
02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
03a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
03b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.272911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.286262,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.134961,NaN,NaN,NaN,NaN,NaN,0.276287,0.327352,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,0.158373,NaN,NaN,NaN,NaN,NaN
05,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.427553,0.000000,NaN,NaN,NaN,NaN,0.236479,0.407576,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,0.300409,0.125191,NaN,0.380054,NaN,0.293185,NaN,0.449998,0.421918,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.127629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.257086,NaN,NaN,NaN,NaN,0.217798,NaN,NaN,NaN,NaN,0.284890,0.172782,NaN,NaN,0.258637,0.145112,0.354909,NaN,NaN,NaN,0.309850,NaN,NaN,0.000000,0.000000,NaN
07,NaN,0.139337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.238929,0.499233,NaN,NaN,0.273251,NaN,NaN,NaN,0.211413,0.000000,0.000000,0.380931,NaN,NaN,NaN,0.102241,0.335018,NaN,NaN,NaN,0.419695,NaN,NaN,NaN,0.000000,0.407920,NaN,NaN,NaN,0.244348,NaN,NaN,NaN,NaN,0.366598,NaN,NaN
07a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.267685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
07b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.250876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df.to_csv('drive/MyDrive/Major proj Colab notebooks/thresholdValues.csv')

In [15]:
pd.read_csv('drive/MyDrive/Major proj Colab notebooks/thresholdValues.csv', index_col=0)

,001,002,003,004,005,006,007,008,009,010,011,012,013,014,015,016,017,018,019,020,021,022,023,024,025,026,027,028,029,030,031,032,033,034,035,036,037,038,039,040,041,042,043,044,045,046,047,048,049,050,051
02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
03a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
03b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.272911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.286262,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.134961,NaN,NaN,NaN,NaN,NaN,0.276287,0.327352,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,0.158373,NaN,NaN,NaN,NaN,NaN
05,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.427553,0.000000,NaN,NaN,NaN,NaN,0.236479,0.407576,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,0.300409,0.125191,NaN,0.380054,NaN,0.293185,NaN,0.449998,0.421918,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.127629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.257086,NaN,NaN,NaN,NaN,0.217798,NaN,NaN,NaN,NaN,0.284890,0.172782,NaN,NaN,0.258637,0.145112,0.354909,NaN,NaN,NaN,0.309850,NaN,NaN,0.000000,0.000000,NaN
07,NaN,0.139337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.238929,0.499233,NaN,NaN,0.273251,NaN,NaN,NaN,0.211413,0.000000,0.000000,0.380931,NaN,NaN,NaN,0.102241,0.335018,NaN,NaN,NaN,0.419695,NaN,NaN,NaN,0.000000,0.407920,NaN,NaN,NaN,0.244348,NaN,NaN,NaN,NaN,0.366598,NaN,NaN
07a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.267685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
07b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.250876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
